In [1]:
import pandas as pd
from os import listdir, mkdir
from os.path import isdir, isfile
import matplotlib.pyplot as plt
from matplotlib.cm import ScalarMappable
import textwrap
import numpy as np
from goatools.obo_parser import GODag
from goatools.gosubdag.gosubdag import GoSubDag
from goatools.godag_plot import plot_gos, plot_goid2goobj #plot_results
from goatools.gosubdag.plot.plot import plot_results
from goatools.go_enrichment import GOEnrichmentRecord
from collections import namedtuple
import os

godag = GODag("../data/goatools_data/go-basic.obo")
pd.options.mode.chained_assignment = None

../data/goatools_data/go-basic.obo: fmt(1.2) rel(2024-10-27) 44,017 Terms


In [2]:
obo_file = "../data/goatools_data/go-basic.obo"

In [3]:
def plot_goa(goafile_enriched:pd.DataFrame,savep:str, filename:str):
    print("[*] Producing plot for {}".format(filename))
    goafile_enriched["ratio_stud"] = goafile_enriched.ratio_in_study.apply(lambda x: int(x.split("/")[0])/int(x.split("/")[1]))
    goafile_enriched["ratio_pop"] = goafile_enriched.ratio_in_pop.apply(lambda x: int(x.split("/")[0])/int(x.split("/")[1]))
    goafile_enriched["amount_in_pop"] = goafile_enriched.ratio_in_pop.apply(lambda x: int(x.split("/")[0]))
    
    categorical_indices = []
    categories = []
    for index, cat in enumerate(list(goafile_enriched.name)):
        if len(cat) >= 30:
            cat = textwrap.fill(cat, width=30)
            categorical_indices.append(index)
        categories.append(cat)
        
    values = list(goafile_enriched.study_count)
    scatter_values = np.array(goafile_enriched.study_count) / np.array(goafile_enriched.amount_in_pop)
    
    pcolors = goafile_enriched.p_fdr_bh
    norm_p_values = np.array(pcolors) / max(pcolors)
    colors=plt.cm.RdBu_r(norm_p_values)
    
    
    # Create figure and axes
    if len(goafile_enriched) == 30:
        fsize = (20,18)
    elif len(goafile_enriched) >= 15:
        fsize = (16,12)
    else:
        fsize = (12,8)
        
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 16), sharey=True)

    # Plot horizontal bar plot on ax1
    ax1.barh(categories, values, color=colors, edgecolor="black")

    ax1.set_xlabel('Count', fontsize=15, labelpad=20)
    ax1.tick_params(axis='y', labelsize=15) 
    ax1.tick_params(axis='x', labelsize=15) 
    
    
    # Accessing tick labels
    tick_labels = ax1.get_yticklabels()

    # Specify indices of labels to make bold (e.g., 1 and 3 in this example)

    # Update tick labels with LaTeX formatting for bold
    for i, label in enumerate(tick_labels):
        if i in categorical_indices:
            #label.set_fontweight('bold')
            label.set_fontsize(10)  # Optional: Adjust font size if needed
            #label.set_color('blue')  # Optional: Adjust font color if needed
            # Use LaTeX for bold formatting
            #label.set_text(r'\textbf{' + label.get_text() + r'}')
    
    
    ax2.scatter(scatter_values, categories, c=colors, cmap='RdBu_r', 
                label='Gene Ratio (compared to Study)', s=list(goafile_enriched.ratio_stud*1000),edgecolor="black")

    ax2.set_xlabel('Count in Study / Count in Pop', fontsize=15, labelpad=20)
    
    ax2.tick_params(axis='x', labelsize=15) 
    #ax1.set_ylabel('GO Categories')
    ax1.invert_yaxis()
    plt.subplots_adjust(left=0.2, wspace=0.1)
    cbar = fig.colorbar(ScalarMappable(cmap='RdBu_r'), ax=[ax1, ax2], pad = 0.005)
    cbar.set_label('p-values',fontsize=15, labelpad=20)
    cbar.set_ticks([min(norm_p_values), max(norm_p_values)])
    cbar.set_ticklabels([f'{min(goafile_enriched.p_fdr_bh):.4f}', f'{max(goafile_enriched.p_fdr_bh):.4f}'])
    cbar.ax.tick_params(labelsize=12)

    cbar.ax.set_position([0.85, 0.15, 0.03, 0.7])
    
    #plt.show()
    plt.savefig(savep + filename + ".svg", dpi=400)
    plt.close()
    print("[*] DONE")

# Transcriptomics

In [7]:
#deseq_df = pd.read_csv("../data/transcriptomics_data/hydra_mono_culture_kiel_vs_liquid_mono_culture_kiel_just_mono.csv")
deseq_df = pd.read_csv("../data/transcriptomics_data/hydra_vs_liquid_mouth_opening_paper.csv",sep=";")
deseq_df.padj = deseq_df.padj.apply(lambda x: float(x.replace(",",".")))
deseq_df.log2FoldChange = deseq_df.log2FoldChange.apply(lambda x: float(x.replace(",",".")))

deseq_df.head()

,locus_tag,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,Length_bp,old_locus_tag
0,AEP_RS02615,"287,5034493",9.006442,"0,30104772","29,91698968","1,18E-196",2.380000e-193,435,AEP_00519
1,AEP_RS17950,"1284,479523",7.684159,"0,348353731","22,05849607","7,92E-108",2.450000e-105,1335,AEP_03593
2,AEP_RS09325,"425,6966443",7.430103,"0,38969423","19,06649408","4,79E-81",6.420000e-79,801,AEP_01870
3,AEP_RS06845,"345,1951382",6.997285,"0,375447772","18,6371725","1,61E-77",1.950000e-75,1155,AEP_01369
4,AEP_RS06820,"98,76114326",6.709044,"0,297051896","22,58542591","6,03E-113",2.200000e-110,1092,AEP_01364


In [8]:
#deseq_df["old_locus_tag"] = deseq_df["Unnamed: 0"].apply(lambda x: x.split("gene:")[-1])
#deseq_df.head()

In [9]:
translation_table = pd.read_table("../data/transcriptomics_data/translation_table_corrected.csv")
translation_table.head()

,protein_id,old_locus_tag,locus_tag
0,WP_087493495.1,AEP_00001,AEP_RS00005
1,WP_087493496.1,AEP_00002,AEP_RS00010
2,WP_087493497.1,AEP_00003,AEP_RS00015
3,WP_087493498.1,AEP_00004,AEP_RS00020
4,WP_087493499.1,AEP_00005,AEP_RS00025


In [10]:
translation_table[~translation_table['old_locus_tag'].isin(deseq_df['old_locus_tag'])]

,protein_id,old_locus_tag,locus_tag
83,WP_087493575.1,AEP_00090,AEP_RS00455
398,WP_087493856.1,AEP_00414,AEP_RS21200
488,WP_087493941.1,AEP_00509,AEP_RS02565
495,WP_257789657.1,AEP_00516,AEP_RS02600
499,WP_087493952.1,AEP_00520,AEP_RS02620
503,WP_087493956.1,AEP_00524,AEP_RS02640
507,WP_087493960.1,AEP_00528,AEP_RS02660
521,WP_087493975.1,AEP_00542,AEP_RS02735
522,WP_087493976.1,AEP_00543,AEP_RS02740
524,WP_087493978.1,AEP_00545,AEP_RS02750


In [11]:
deseq_df = deseq_df.merge(translation_table, on="old_locus_tag")

In [12]:
deseq_df = deseq_df[deseq_df["padj"] <= 0.05]
upregulated_genes = deseq_df[deseq_df["log2FoldChange"] >= 1.0]
downregulated_genes = deseq_df[deseq_df["log2FoldChange"] <= -1.0]

In [13]:
downregulated_genes.sort_values(by="log2FoldChange",ascending=True)[:10]

,locus_tag_x,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,Length_bp,old_locus_tag,protein_id,locus_tag_y
3876,AEP_RS05165,"464,7806893",-4.600618,"0,38258511","-12,02508337","2,62E-33",6.800000e-32,303,AEP_01026,WP_087494398.1,AEP_RS05165
3875,AEP_RS04110,"1100,651052",-4.542414,"0,341237229","-13,31160067","1,98E-40",7.650000e-39,270,AEP_00816,WP_066709320.1,AEP_RS04110
3874,AEP_RS11225,"1722,675962",-4.439811,"0,343126042","-12,93930107","2,70E-38",8.820000e-37,270,AEP_02252,WP_157673145.1,AEP_RS11225
3873,AEP_RS11245,"819,1336456",-4.311283,"0,223596993","-19,28149024","7,68E-83",1.060000e-80,264,AEP_02256,WP_087495464.1,AEP_RS11245
3872,AEP_RS11240,"1220,324739",-4.258587,"0,260157439","-16,36926968","3,17E-60",2.600000e-58,342,AEP_02255,WP_157673146.1,AEP_RS11240
3871,AEP_RS11230,"4126,155348",-4.224712,"0,316463591","-13,34975755","1,19E-40",4.680000e-39,843,AEP_02253,WP_232459812.1,AEP_RS11230
3870,AEP_RS08130,"2623,269159",-3.922157,"0,353675551","-11,08970245","1,41E-28",2.700000e-27,441,AEP_01625,WP_087494915.1,AEP_RS08130
3869,AEP_RS11255,"2894,556421",-3.881258,"0,184214776","-21,06920145","1,52E-98",3.380000e-96,822,AEP_02258,WP_232460007.1,AEP_RS11255
3868,AEP_RS11250,"2948,841537",-3.744059,"0,213236646","-17,55823634","5,14E-69",5.040000e-67,912,AEP_02257,WP_087495465.1,AEP_RS11250
3867,AEP_RS11025,"469,8090314",-3.683200,"0,290943199","-12,65951711","9,91E-37",2.930000e-35,522,AEP_02210,WP_087495426.1,AEP_RS11025


In [14]:
upregulated_genes.sort_values(by="log2FoldChange",ascending=False)[:40]

,locus_tag_x,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,Length_bp,old_locus_tag,protein_id,locus_tag_y
0,AEP_RS02615,"287,5034493",9.006442,"0,30104772","29,91698968","1,18E-196",2.380000e-193,435,AEP_00519,WP_087493951.1,AEP_RS02615
1,AEP_RS17950,"1284,479523",7.684159,"0,348353731","22,05849607","7,92E-108",2.450000e-105,1335,AEP_03593,WP_087496666.1,AEP_RS17950
2,AEP_RS09325,"425,6966443",7.430103,"0,38969423","19,06649408","4,79E-81",6.420000e-79,801,AEP_01870,WP_087495130.1,AEP_RS09325
3,AEP_RS06845,"345,1951382",6.997285,"0,375447772","18,6371725","1,61E-77",1.950000e-75,1155,AEP_01369,WP_087494696.1,AEP_RS06845
4,AEP_RS06820,"98,76114326",6.709044,"0,297051896","22,58542591","6,03E-113",2.200000e-110,1092,AEP_01364,WP_087494692.1,AEP_RS06820
5,AEP_RS00810,"205,2457755",6.601383,"0,300195431","21,99028415","3,57E-107",1.020000e-104,399,AEP_00159,WP_087493637.1,AEP_RS00810
6,AEP_RS06695,"380,9184638",6.451893,"0,278516947","23,16517105","1,02E-118",4.560000e-116,372,AEP_01339,WP_232459951.1,AEP_RS06695
7,AEP_RS13420,"1526,88337",6.091125,"0,405220734","15,03162274","4,56E-51",2.860000e-49,1356,AEP_02694,WP_198301826.1,AEP_RS13420
8,AEP_RS03285,"458,889393",6.005292,"0,230505254","26,05273478","1,25E-149",7.190000e-147,1323,AEP_00654,WP_157673027.1,AEP_RS03285
9,AEP_RS18030,"45,0584585",5.992117,"0,418041334","14,33379116","1,35E-46",7.500000e-45,912,AEP_03608,WP_087497409.1,AEP_RS18030


In [15]:
assoc_file = "../data/transcriptomics_data/goatools_data/associations_06.txt"
pop_file = "../data/transcriptomics_data/goatools_data/p_06.txt"

goatools_output = "../results/processed_data/goatools_output/goatools_results_06_up.txt"

In [16]:
proteins = []
with open(pop_file,"r") as popfile:
    for line in popfile.readlines():
        proteins.append(line.strip())

In [17]:
with open("../data/transcriptomics_data/goatools_data/upregulated_proteins_sample_06.txt", "w") as outfile:
    for protein in proteins:
        if protein in list(upregulated_genes["protein_id"]):
            outfile.write(protein+"\n")

In [18]:
with open("../data/transcriptomics_data/goatools_data/downregulated_proteins_sample_06.txt", "w") as outfile:
    for protein in proteins:
        if protein in list(downregulated_genes["protein_id"]):
            outfile.write(protein+"\n")

In [19]:
sam_file = "../data/transcriptomics_data/goatools_data/upregulated_proteins_sample_06.txt"

In [20]:
!find_enrichment.py $sam_file $pop_file $assoc_file --annofmt id2gos --alpha 0.05 --pval 0.05 --obo $obo_file --method fdr_bh --outfile $goatools_output --obsolete replace

../data/goatools_data/go-basic.obo: fmt(1.2) rel(2024-10-27) 51,566 Terms; optional_attrs(consider replaced_by)
HMS:0:00:00.183653  28,814 annotations READ: ../data/transcriptomics_data/goatools_data/associations_06.txt 
Study: 585 vs. Population 2562


Load BP Ontology Enrichment Analysis ...
Propagating term counts up: is_a
 70%  1,801 of  2,562 population items found in association

Load CC Ontology Enrichment Analysis ...
Propagating term counts up: is_a
 17%    437 of  2,562 population items found in association

Load MF Ontology Enrichment Analysis ...
Propagating term counts up: is_a
 87%  2,217 of  2,562 population items found in association

Runing BP Ontology Analysis: current study set of 585 IDs.
 62%    362 of    585 study items found in association
100%    585 of    585 study items found in population(2562)
Calculating 1,917 uncorrected p-values using fisher_scipy_stats
   1,917 terms are associated with  1,801 of  2,562 population items
     945 terms are associated with

In [21]:
sam_file = "../data/transcriptomics_data/goatools_data/downregulated_proteins_sample_06.txt"
goatools_output = "../results/processed_data/goatools_output/goatools_results_06_down.txt"

In [22]:
!find_enrichment.py $sam_file $pop_file $assoc_file --annofmt id2gos --alpha 0.05 --pval 0.05 --obo $obo_file --method fdr_bh --outfile $goatools_output --obsolete replace

../data/goatools_data/go-basic.obo: fmt(1.2) rel(2024-10-27) 51,566 Terms; optional_attrs(consider replaced_by)
HMS:0:00:00.114022  28,814 annotations READ: ../data/transcriptomics_data/goatools_data/associations_06.txt 
Study: 476 vs. Population 2562


Load BP Ontology Enrichment Analysis ...
Propagating term counts up: is_a
 70%  1,801 of  2,562 population items found in association

Load CC Ontology Enrichment Analysis ...
Propagating term counts up: is_a
 17%    437 of  2,562 population items found in association

Load MF Ontology Enrichment Analysis ...
Propagating term counts up: is_a
 87%  2,217 of  2,562 population items found in association

Runing BP Ontology Analysis: current study set of 476 IDs.
 77%    366 of    476 study items found in association
100%    476 of    476 study items found in population(2562)
Calculating 1,917 uncorrected p-values using fisher_scipy_stats
   1,917 terms are associated with  1,801 of  2,562 population items
   1,020 terms are associated with

In [23]:
# plotting upregulated terms
goatools_output = "../results/processed_data/goatools_output/goatools_results_06_up.txt"
dataframe = pd.read_csv(goatools_output,sep="\t")
dataframe_sig = dataframe[dataframe["p_fdr_bh"] <= 0.05]
enriched_sig = dataframe[dataframe["enrichment"] == "e"]
purified_sig = dataframe[dataframe["enrichment"] == "p"]

plot_goa(purified_sig, "../results/figures/goatools/", "purified_upregulated_terms")
plot_goa(enriched_sig, "../results/figures/goatools/", "enriched_upregulated_terms")

[*] Producing plot for purified_upregulated_terms


/tmp/ipykernel_5693/2154827721.py:56: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  ax2.scatter(scatter_values, categories, c=colors, cmap='RdBu_r',


[*] DONE
[*] Producing plot for enriched_upregulated_terms
[*] DONE


In [25]:
# plotting upregulated terms
goatools_output = "../results/processed_data/goatools_output/goatools_results_06_down.txt"
dataframe = pd.read_csv(goatools_output,sep="\t")
dataframe_sig = dataframe[dataframe["p_fdr_bh"] <= 0.05]
enriched_sig = dataframe[dataframe["enrichment"] == "e"]
#purified_sig = dataframe[dataframe["enrichment"] == "p"]

#plot_goa(purified_sig, "../results/figures/goatools/", "purified_downregulated_terms")
plot_goa(enriched_sig, "../results/figures/goatools/", "enriched_downregulated_terms")

[*] Producing plot for enriched_downregulated_terms


/tmp/ipykernel_5693/2154827721.py:56: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  ax2.scatter(scatter_values, categories, c=colors, cmap='RdBu_r',


[*] DONE
